In [35]:
import numpy as np
import tensorflow as tf
import os
import pickle
import matplotlib.pyplot as plt
from scipy import misc
from scipy.spatial import distance

%matplotlib inline

# facenetのダウンロード
- https://github.com/davidsandberg/facenet
- クローンしたパスをfacenet_pathに
- tensorflowのversionは1.2が必要

# 訓練済みのモデルのダウンロード
- [MS-Celeb-1Mで訓練したモデル](https://drive.google.com/file/d/0B5MzpY9kBtDVOTVnU3NIaUdySFE/edit)
- ダウンロードしたパスをmodelに

In [36]:
facenet_path = '../../facenet/'
model = facenet_path + 'model/20170511-185253'
image_dir_basepath = './'

In [37]:
import sys
sys.path.append(facenet_path + 'src/')
import facenet
import align.detect_face

from PIL import Image

# 顔検出

In [38]:
image_size = 160
margin = 44
batch_size = 1000

In [46]:
talents = ['takemoto','taniai', 'suzuki']

for talent in talents:
    
    dir_name = 'images_{}/'.format(talent)
    image_dirpath = os.path.abspath(image_dir_basepath + dir_name)
    image_filepaths = [os.path.join(image_dirpath, f) for f in os.listdir(image_dirpath) if not f == '.DS_Store']

    image_out_dirpath = './cropped_images_{}/'.format(talent)
    if not os.path.exists(image_out_dirpath):
            os.mkdir(image_out_dirpath)

    for image_filepath in image_filepaths:
        extracted_filepaths = []
        minsize = 20
        threshold = [0.6, 0.7, 0.7]
        factor = 0.709

        with tf.Graph().as_default():
            with tf.Session() as sess:
                pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)
                img = misc.imread(image_filepath)
                out_file_name = "crop_" + os.path.basename(image_filepath)
                img_size = np.array(img.shape)[0:2]
                bounding_boxes, _ = align.detect_face.detect_face(img, minsize, pnet, rnet, onet,
                                                                  threshold, factor)
                det = bounding_boxes[0, :4]
                bb = np.zeros(4, dtype=np.int32)
                bb[0] = np.maximum(det[0]-margin/2, 0)
                bb[1] = np.maximum(det[1]-margin/2, 0)
                bb[2] = np.minimum(det[2]+margin/2, img_size[1])
                bb[3] = np.minimum(det[3]+margin/2, img_size[0])
                cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
                aligned = misc.imresize(cropped, (image_size, image_size), interp='bilinear')

                img = Image.fromarray(np.uint8(aligned))
                img.save(os.path.join(image_out_dirpath,out_file_name))

                #test (try only on one image)
                # plt.figure(figsize=(20, 10))
                # plt.subplot(1,3,1)
                # plt.imshow(img)
                # plt.subplot(1,3,2)
                # plt.imshow(cropped)
                # plt.subplot(1,3,3)
                # plt.imshow(aligned)